# Medical Chatbot

Our goal is to have a lightweight and medically accurate chatbot that answers queries precisely. For that purpose, we fine-tune 3 small-sized LLMs on our custom curated dataset using Unsloth library with LoRA.
Here, we use 3 different models with different parameter sizes, specifically:
1. `llama-3.2-1B-Instruct`
2. `llama-3.2-3B-Instruct`
3. `qwen-2.5-7B`

This notebook only focuses on the first model. Other two notebooks focus on the other remaining notebooks respectively.

## Setup

In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import get_chat_template

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


#### Getting our Dataset from GitHub:

In [ ]:
!wget https://raw.githubusercontent.com/farazahmad2004/NLP_Project_Medical_Chatbot/main/datasets/Roman_Urdu/jsonl/train_dataset.jsonl
!wget https://raw.githubusercontent.com/farazahmad2004/NLP_Project_Medical_Chatbot/main/datasets/Roman_Urdu/jsonl/val_dataset.jsonl

--2025-12-13 10:33:22--  https://raw.githubusercontent.com/farazahmad2004/NLP_Project_Medical_Chatbot/main/datasets/Roman_Urdu/jsonl/train_dataset.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20532209 (20M) [text/plain]
Saving to: ‘train_dataset.jsonl’

train_dataset.jsonl 100%[===================>]  19.58M  --.-KB/s    in 0.08s   

2025-12-13 10:33:22 (261 MB/s) - ‘train_dataset.jsonl’ saved [20532209/20532209]

--2025-12-13 10:33:22--  https://raw.githubusercontent.com/farazahmad2004/NLP_Project_Medical_Chatbot/main/datasets/Roman_Urdu/jsonl/val_dataset.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|1

## Model 1 (`llama-3.2-1B-Instruct`) Setup & Tokenizer:

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

# 1. loading llama-3.2-1B-Instruct
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
# 2. adding LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth 2025.11.6 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


### Format Dataset:

In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts }

dataset = load_dataset("json", data_files="train_dataset.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/21150 [00:00<?, ? examples/s]

## Training:

Maximum training steps were set to 2600 which correspond to around 1 epoch of fine-tuning on our whole dataset. We don't do more fine-tuning to avoid catastrophic forgetting as our goal here is just to make sure that the model learns conversing in roman urdu and some medical terms.

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 2600,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/21150 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 21,150 | Num Epochs = 1 | Total steps = 2,600
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 11,272,192 of 1,247,086,592 (0.90% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.700200
2,3.810400
3,3.710800
4,3.559000
5,3.587200
6,3.347900
7,2.989200
8,2.982200
9,2.881900
10,2.769300


TrainOutput(global_step=2600, training_loss=1.7159967206991635, metrics={'train_runtime': 4558.0533, 'train_samples_per_second': 4.563, 'train_steps_per_second': 0.57, 'total_flos': 4.871487300199219e+16, 'train_loss': 1.7159967206991635, 'epoch': 0.983451536643026})

## Inference (just for small testing):

In [ ]:
FastLanguageModel.for_inference(model)
messages = [
    {"role": "user", "content": "mujhe bukhar hai aur sardi lag rahi hai"}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")
outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True, temperature=0.6, min_p=0.1, repetition_penalty=1.2)
print(tokenizer.batch_decode(outputs))

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nmujhe bukhar hai aur sardi lag rahi hai<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\naapko apne doctor se milna chahiye taake wo aapke asraat ka jaiza le sakein. agar aisa ho to kisi ko bhi nazar rakho, phir sirf un logon ke liye jo iski wajah samjha jaye.<|eot_id|>']


In [ ]:
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "meray galey mein kaafi dard ho raha hai, aur cheenkein bhe aa rahi hein, mujhe kia karna chahye?"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")
outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True, temperature=0.6, min_p=0.1, repetition_penalty=1.2)
print(tokenizer.batch_decode(outputs))

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nmeray galey mein kaafi dard ho raha hai, aur cheenkein bhe aa rahi hein, mujhe kia karna chahye?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\naapko apne doctor se milna hoga taake wo aapki madad kar sakein. aapka jism is maslay ke liye ek maqam par tayyar nikaal sakta hai jab tak aisa hota hai ke aapko ye asraat mehsoos na hone diyein. agar aap in halaton ko behtar samjha sakte hain to aapko ye tasalli dekhni hogi. shukriya.<|eot_id|>']


# Saving the model:

## On HuggingFace:

In [ ]:
from huggingface_hub import login
# login(token="") # add your here
repo_name = "farazahmad2004/NLP-Medical-Chatbot-Llama-1B"
model.push_to_hub(repo_name, token=True)
tokenizer.push_to_hub(repo_name, token=True)
print(f"Model pushed to: https://huggingface.co/{repo_name}")

# Getting the model from HuggingFace:

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "farazahmad2004/NLP-Medical-Chatbot-Llama-1B",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.12.5: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

Unsloth 2025.12.5 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

# Evaluation: (initial and only for 50 queries)

## Evaluation Setup:



In [ ]:
!pip install evaluate rouge_score bert_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=509cffd84aac88a38c480a01df1a0f576d4b651848b9faf5720ba7fd7bbe4df3
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [ ]:
import evaluate
from tqdm import tqdm

## Validation Dataset:

In [ ]:
VAL_FILE = "val_dataset.jsonl"
NUM_SAMPLES = 50

dataset = load_dataset("json", data_files=VAL_FILE, split="train")
if NUM_SAMPLES > 0:
    dataset = dataset.shuffle(seed=42).select(range(NUM_SAMPLES))

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
rouge = evaluate.load('rouge')
bertscore = evaluate.load('bertscore')

In [ ]:
def generate_response(question):
    messages = [{"role": "user", "content": question}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=128,
        use_cache=True,
        temperature=0.6,
        repetition_penalty=1.2
    )
    generated_tokens = outputs[0][inputs.shape[1]:]
    decoded_response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return decoded_response.strip()

In [ ]:
print(f"Generating answers for {len(dataset)} samples...")
generated_answers = []
ground_truths = []
for row in tqdm(dataset):
    user_q = row['messages'][1]['content']
    true_a = row['messages'][2]['content']
    pred_a = generate_response(user_q)
    generated_answers.append(pred_a)
    ground_truths.append(true_a)

# rouge-l
print("\n--- Calculating Metrics ---")
rouge_results = rouge.compute(predictions=generated_answers, references=ground_truths)
print(f"ROUGE-L: {rouge_results['rougeL']:.4f}")
# bert score
bert_results = bertscore.compute(predictions=generated_answers, references=ground_truths, lang="en")
print(f"BERTScore (F1): {sum(bert_results['f1']) / len(bert_results['f1']):.4f}")
print("\n--- Example Outputs ---")
for i in range(5):
    print(f"Q: {dataset[i]['messages'][1]['content']}")
    print(f"True: {ground_truths[i]}")
    print(f"Pred: {generated_answers[i]}")
    print("-" * 30)

Generating answers for 50 samples...


100%|██████████| 50/50 [04:01<00:00,  4.83s/it]



--- Calculating Metrics ---
ROUGE-L: 0.1195


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore (F1): 0.8464

--- Example Outputs ---
Q: kya aap meri madad kar sakte hain samajhne mein ke mera boyfriend kyun ejaculate nahi kar pa raha
True: shayad usay retrograde ejaculation hai jahan sperm uski bladder mein wapas chalay jate hain ya shayad wo sirf urethra mein fluid nahi daal raha. hum yeh kuch surgeries ya kuch dawaiyon ke baad dekhte hain lekin yeh ek sehatmand aadmi ke liye bina kisi wajah ke aam nahi hai. mujhe lagta hai usay ek urologist se milna chahiye taake yeh confirm ho sake ke sab theek hai, yeh zaroori hai.
Pred: mujhe afsos hai lekin yeh ek mumkinat kaafi baatein se zyada ho sakta hai. is par bohot si research ki gayi hai aur in studies ko dekhte hue, humein pata chala ke hum apni sex life bhi jaldi karne wale hote hain. agar aapka boyfriend us waqt nahi tha jab wo ejaculation shuru kar raha tha to usay yaqeen dilana zaroori hai. maine kabhi bhi aise jawab diye the jo mujhe khushi dene lagi. ek study
------------------------------
Q: kya aap gandi haath se

## Human Evaluation:

In [ ]:
symptoms_questions = [
    "Mujhe subah se sar mein dard ho raha hai, koi dawai bataen?",
    "Mere bachay ko tez bukhar hai aur wo kuch kha nahi raha.",
    "Mujhe khansi ke sath balgham aa raha hai, kya karoon?",
    "Pait mein mror uth rahe hain aur loose motion lage hain.",
    "Mere daant mein shadeed dard hai, dentist ke paas janay tak kya karoon?",
    "Mujhe saans lene mein dushwari ho rahi hai jab main sidhiyan charhta hoon.",
    "Meri aankhon mein jalan aur pani aa raha hai.",
    "Kaan mein dard hai, shayed paani chala gaya hai.",
    "Jism par lal nishan par gaye hain aur kharish ho rahi hai.",
    "Mujhe ulti jaisa mehsoos ho raha hai (nausea). kia karun?"
]
for question in symptoms_questions:
    print(f"Q: {question}")
    print(f"A: {generate_response(question=question)}")
    print("-"*70)

Q: Mujhe subah se sar mein dard ho raha hai, koi dawai bataen?
A: Aapko apne doctor ke paas jana chahiye taake wo aapke asraat ka pata laga sake aur isay ilaj kar saken. Aam tor par yeh kuch infection ki wajah se hota hai jo ek x-ray ya CT scan se theek kiya ja sakta hai. Agar sab kuch theek hai to aapko apni zindagi ko normal nikaalna hoga. Shukriya.
----------------------------------------------------------------------
Q: Mere bachay ko tez bukhar hai aur wo kuch kha nahi raha.
A: Yeh sirf viral infection ki wajah se ho sakta hai, isliye aapko foran medical madad leni chahiye taake wo theek ho sakein. Agar yeh zyada tar ek acute bacterial pharyngitis ka asal sabab banta hai to antibiotic treatment shuru karne ke liye zaroorat hogi. Aapka doctor antibiotics lene par ghor kar sakta hai ya agar mumkin ho to doosri dawaiyan jaise antihistamines le sakte hain. Iske ilawa, achhi neend la
----------------------------------------------------------------------
Q: Mujhe khansi ke sath balgham 